In [1]:
import numpy as np
import pandas as pd
import os
import lightgbm as lgb
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

from google.colab import drive
from datetime import datetime

In [2]:
drive.mount('/content/drive')
root_path = 'drive/My Drive/KU/Robin_lab_project' 
os.chdir(root_path)

Mounted at /content/drive


# Data exploration

In [ ]:
# Load data 
profiles_all = pd.read_csv("Data/ML_input/profiles_all_replicate_negfiltered.csv")
metadata_all = pd.read_csv("Data/ML_input/metadata_all_replicate_negfiltered.csv")

X = profiles_all.drop("label", axis=1)
X = StandardScaler().fit_transform(X)
y = profiles_all["label"]

print(X.shape)
print(y.shape)

In [ ]:
#  PCA plot
def multi_plot_pca(x, y, 
                   nrows=2, xlim=(-23,43), ylim=(-23,23),
                   figsize=(12,7), top=0.89,
                   title="", save=False, filename="pca"):
  # Perform PCA and MDS
  n_pcs = nrows*3+1
  pca = PCA(n_components=n_pcs)
  pcs = pca.fit_transform(x)
  var = pca.explained_variance_ratio_ * 100
  tmp_df = pd.concat([pd.DataFrame(pcs), y], axis = 1)
  targets = [1,0]
  colors = ["r","g"]
  # Plot fig and axes
  fig, axes = plt.subplots(nrows, 3, figsize = figsize, sharex=True, sharey=True)
  fig.add_subplot(111, frameon=False)
  for target, color in zip(targets,colors):
    i_keep = tmp_df['label'] == target
    for i ,ax in enumerate(axes.flatten()):
      ax.scatter(tmp_df.loc[i_keep,i], tmp_df.loc[i_keep,i+1], zorder=3, 
                      ec="black", c=color, s=25, alpha = 0.8, label = target) 
      ax.set_title(f"PC{i+1} ({var[i]:.2}%) and PC{i+2} ({var[i+1]:.2}%)")
      ax.set_xlim(xlim)
      ax.set_ylim(ylim)
  # Details
  legend = ax.legend(frameon = 1, shadow = True, bbox_to_anchor=(1.3, 1.25))
  frame = legend.get_frame()
  frame.set_facecolor('white')
  frame.set_edgecolor('black')
  plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
  fig.suptitle(f"PCA first {n_pcs} PCs{title}", fontsize=16)
  plt.xlabel("First (relative) component", fontsize=13)
  plt.ylabel("Second (relative) component", fontsize=13)
  fig.tight_layout()
  fig.subplots_adjust(top=top)
  if save == True:
      plt.savefig(f"{filename}.png", dpi = 300)
  plt.show()

multi_plot_pca(X, y, nrows=3, figsize=(12,12), top=0.92, 
               save=True, filename="Plots/pca_multiplot_allchr_all_replicate_negFiltered")     

# Models development

### Split train and test by chromosomes number

In [ ]:
# Load data divded by chr number
profiles_train = pd.read_csv("Data/ML_input/profiles_all_replicate_negfiltered_train.csv")
profiles_test = pd.read_csv("Data/ML_input/profiles_all_replicate_negfiltered_test.csv")
metadata_train = pd.read_csv("Data/ML_input/metadata_all_replicate_negfiltered_train.csv")
metadata_test = pd.read_csv("Data/ML_input/metadata_all_replicate_negfiltered_test.csv")

# Divide predictor and target
X_TRAIN = profiles_train.drop("label", axis = 1)
y_TRAIN = profiles_train["label"]
X_TEST = profiles_test.drop("label", axis = 1)
y_TEST = profiles_test["label"]

# # Normalization
# scaler = StandardScaler()
# X_TRAIN = scaler.fit_transform(X_TRAIN)
# X_TEST = scaler.transform(X_TEST)

print(X_TRAIN.shape)
print(y_TRAIN.shape)
print(X_TEST.shape)
print(y_TEST.shape)

In [ ]:
def get_accuracy(ypred, y):
  return sum(ypred == y) / len(y)

def rmse(yprob, y):
  return np.sqrt(np.mean((yprob-y)**2))


##### Randomly split TRAIN into train and validation keeping labels proportion (it can be use as direct validation in alternative to CV)

In [ ]:
# Split training set into train and validation
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=20)
sss.split(X_TRAIN, y_TRAIN)

for itrain, ival in sss.split(X_TRAIN, y_TRAIN):
  X_train, X_val = X_TRAIN.iloc[itrain], X_TRAIN.iloc[ival]
  y_train, y_val = y_TRAIN[itrain], y_TRAIN[ival]

# Normalization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

## Random forest

In [ ]:
def rf_train_pred(xtrain, ytrain, xtest, ytest, n_tree=100, name="RF"):
  start_time = datetime.now()
  # Train
  model = RandomForestClassifier(n_estimators=n_tree, bootstrap=True)
  model.fit(xtrain, ytrain)
  # Predict
  ytrain_pred = model.predict(xtrain)
  ytrain_prob = model.predict_proba(xtrain)[:,1]
  ytest_pred = model.predict(xtest)
  ytest_prob = model.predict_proba(xtest)[:,1]
  # Evaluate
  print(f"{name} train accuracy: {get_accuracy(ytrain_pred, ytrain):.4}")
  print(f"{name} test accuracy: {get_accuracy(ytest_pred, ytest):.4}")
  # Report time 
  print(f"Duration: {datetime.now() - start_time}")
  return model, ytrain_prob, ytrain_pred, ytest_prob, ytest_pred,

rf_model, rf_train_yprob, rf_train_ypred, rf_val_yprob, rf_val_ypred = rf_train_pred(X_train, y_train, X_val, y_val)   
#0.6559 (real test)  # 0.67624 (splitted train into train and val) 

In [ ]:
# Random forest CV
def rf_cv(xtrain, ytrain, n_tree=100, kfold=10, name="RF"):
  start_time = datetime.now()
  print(f"Performing {name} CV")
  # Initialize arrays to store predictions and true values
  train_yprob_vec, train_ypred_vec, train_ytrue_vec = np.array([]), np.array([]), np.array([])
  val_yprob_vec, val_ypred_vec, val_ytrue_vec = np.array([]), np.array([]), np.array([])
  # Stratified CV
  skf = StratifiedKFold(n_splits=kfold, shuffle=True, random_state=26)
  for i, (itrain, ival) in enumerate(skf.split(xtrain, ytrain)):
    print(f"\n> Starting CV iteration {i+1}")
    # Split data according to folds
    X_train, X_val = xtrain.iloc[itrain], xtrain.iloc[ival]
    y_train, y_val = ytrain.iloc[itrain], ytrain.iloc[ival]
    # Normalization
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    # Train and predict
    _, train_yprob, train_ypred, val_yprob, val_ypred = rf_train_pred(X_train, y_train, X_val, y_val, n_tree)
    # Store prediction on training data
    train_yprob_vec = np.concatenate((train_yprob_vec, train_yprob))
    train_ypred_vec = np.concatenate((train_ypred_vec, train_ypred))
    train_ytrue_vec = np.concatenate((train_ytrue_vec, y_train))
    # Store prediction on validation data
    val_yprob_vec = np.concatenate((val_yprob_vec, val_yprob))
    val_ypred_vec = np.concatenate((val_ypred_vec, val_ypred))
    val_ytrue_vec = np.concatenate((val_ytrue_vec, y_val))
  # Stack training and validation predictions into two panda df
  train_output = pd.DataFrame({"yprob": train_yprob_vec, "ypred": train_ypred_vec,"ytrue": train_ytrue_vec})
  val_output = pd.DataFrame({"yprob": val_yprob_vec, "ypred": val_ypred_vec,"ytrue": val_ytrue_vec})
  # Evaluate
  train_acc = get_accuracy(train_output["ypred"].values, train_output["ytrue"].values)
  val_acc = get_accuracy(val_output["ypred"].values, val_output["ytrue"].values)
  print(f"\n>> {name} final report")
  print(f"Train CV accuracy: {train_acc:.4}")
  print(f"Valid CV accuracy: {val_acc:.4}")
  # Report time 
  print(f"Duration: {datetime.now() - start_time}")
  return train_output, val_output

rf_train_output, rf_val_output = rf_cv(X_TRAIN, y_TRAIN)
display(rf_val_output)

## LightGBM    
### >> Should validate on validation data <<
Should split train into training and validation keeping both labels and chr proportion 

In [ ]:
def lgb_train_pred(xtrain, ytrain, xtest, ytest, par, name="LGBM"):
  start_time = datetime.now()
  # Train
  train_data = lgb.Dataset(xtrain , label = ytrain)
  valid_data = lgb.Dataset(xtest, label = ytest)
  model = lgb.train(par, train_data, valid_sets=[valid_data], verbose_eval=20) 
  # Predict
  ytrain_prob = model.predict(np.array(xtrain))
  ytrain_pred = np.round(ytrain_prob)
  ytest_prob = model.predict(np.array(xtest))
  ytest_pred = np.round(ytest_prob)
  # Evaluate
  print(f"{name} train accuracy: {get_accuracy(ytrain_pred, ytrain):.4}")
  print(f"{name} test accuracy: {get_accuracy(ytest_pred, ytest):.4}")
  # Report time 
  print(f"Duration: {datetime.now() - start_time}")
  return model, ytrain_prob, ytrain_pred, ytest_prob, ytest_pred

params = {"application" : "binary",
          "num_boost_round" : 400,
         # "metric" :"binary_logloss",
          "metric" :"rmse",
          "force_row_wise" : True,
          "learning_rate" : 0.009,            
         # "sub_feature" : 0.8,
         # "sub_row" : 0.75,
         # "bagging_freq" : 1,
         # "lambda_l2" : 0.1,
         # 'verbosity': 1,
          'num_iterations' : 1500
         # 'num_leaves': 128,
         # "min_data_in_leaf": 100,
}

lgb_model, lgb_train_yprob, lgb_train_ypred, lgb_val_yprob, lgb_val_ypred = lgb_train_pred(X_train, y_train, X_val, y_val, params)

In [ ]:
# LGBM CV
def lgb_cv(xtrain, ytrain, par, kfold=10, name="LGBM"):
  start_time = datetime.now()
  print(f"Performing {name} CV")
  # Initialize 1d df to store predictions and true values
  train_yprob_vec, train_ypred_vec, train_ytrue_vec = np.array([]), np.array([]), np.array([])
  val_yprob_vec, val_ypred_vec, val_ytrue_vec = np.array([]), np.array([]), np.array([])
  # Stratified CV
  skf = StratifiedKFold(n_splits=kfold, shuffle=True, random_state=26)
  for i, (itrain, ival) in enumerate(skf.split(xtrain, ytrain)):
    print(f"\n> Starting CV iteration {i+1}")
    # Split data according to folds
    X_train, X_val = xtrain.iloc[itrain], xtrain.iloc[ival]
    y_train, y_val = ytrain.iloc[itrain], ytrain.iloc[ival]
    # Normalization
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    # Train and predict
    _, train_yprob, train_ypred, val_yprob, val_ypred = lgb_train_pred(X_train, y_train, X_val, y_val, par=par)
    # Store prediction on training data
    train_yprob_vec = np.concatenate((train_yprob_vec, train_yprob))
    train_ypred_vec = np.concatenate((train_ypred_vec, train_ypred))
    train_ytrue_vec = np.concatenate((train_ytrue_vec, y_train))
    # Store prediction on validation data
    val_yprob_vec = np.concatenate((val_yprob_vec, val_yprob))
    val_ypred_vec = np.concatenate((val_ypred_vec, val_ypred))
    val_ytrue_vec = np.concatenate((val_ytrue_vec, y_val))
  # Stack training and validation predictions into two panda df
  train_output = pd.DataFrame({"yprob": train_yprob_vec, "ypred": train_ypred_vec,"ytrue": train_ytrue_vec})
  val_output = pd.DataFrame({"yprob": val_yprob_vec, "ypred": val_ypred_vec,"ytrue": val_ytrue_vec})
  # Evaluate
  train_acc = get_accuracy(train_output["ypred"].values, train_output["ytrue"].values)
  val_acc = get_accuracy(val_output["ypred"].values, val_output["ytrue"].values)
  print(f"\n>> {name} final report")
  print(f"Train CV accuracy: {train_acc:.4}")
  print(f"Valid CV accuracy: {val_acc:.4}")
  # Report time 
  print(f"Duration: {datetime.now() - start_time}")
  return train_output, val_output

lgb_train_output, lgb_val_output = lgb_cv(X_TRAIN, y_TRAIN, params)

In [ ]:
# # LGBM Feature importance
# plt.rcParams['figure.figsize'] = (18.0, 4)
# fig, ax = plt.subplots(figsize=(12,8))
# lgb.plot_importance(lgb_model, max_num_features=50, height=0.8, ax=ax)
# ax.grid(False)
# plt.title("LightGBM - Feature Importance ", fontsize=15)

## SVM

In [ ]:
def svm_train_pred(xtrain, ytrain, xtest, ytest, model, name="SVM"):
  start_time = datetime.now()
  # Train and predict
  model = model.fit(xtrain, ytrain)
  ytrain_prob = model.predict(xtrain)
  ytest_prob = model.predict(xtest)
  ytrain_pred = np.round(ytrain_prob)
  ytest_pred = np.round(ytest_prob)
  # Evaluate
  print(f"{name} train accuracy: {get_accuracy(ytrain_pred, ytrain)}")
  print(f"{name} test accuracy: {get_accuracy(ytest_pred, ytest)}")
  # Report time 
  print(f"Duration: {datetime.now() - start_time}")
  return model, ytrain_prob, ytrain_pred, ytest_prob, ytest_pred

svm_rbf = svm.SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
svm_model, svm_train_yprob, svm_train_ypred, svm_test_yprob, svm_test_ypred = svm_train_pred(X_train, y_train, X_val, y_val, model=svm_rbf)

In [ ]:
# SVM CV
def svm_cv(xtrain, ytrain, model, kfold=10, name="SVM"):
  start_time = datetime.now()
  print(f"Performing {name} CV")
  # Initialize 1d df to store predictions and true values
  train_yprob_vec, train_ypred_vec, train_ytrue_vec = np.array([]), np.array([]), np.array([])
  val_yprob_vec, val_ypred_vec, val_ytrue_vec = np.array([]), np.array([]), np.array([])
  # Stratified CV
  skf = StratifiedKFold(n_splits=kfold, shuffle=True, random_state=26)
  for i, (itrain, ival) in enumerate(skf.split(xtrain, ytrain)):
    print(f"\n> Starting CV iteration {i+1}")
    # Split data according to folds
    X_train, X_val = X_TRAIN.iloc[itrain], X_TRAIN.iloc[ival]
    y_train, y_val = y_TRAIN[itrain], y_TRAIN[ival]
    # Normalization
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    # Train and predict
    _, train_yprob, train_ypred, val_yprob, val_ypred = svm_train_pred(X_train, y_train, X_val, y_val, model=model)
    # Store prediction on training data
    train_yprob_vec = np.concatenate((train_yprob_vec, train_yprob))
    train_ypred_vec = np.concatenate((train_ypred_vec, train_ypred))
    train_ytrue_vec = np.concatenate((train_ytrue_vec, y_train))
    # Store prediction on validation data
    val_yprob_vec = np.concatenate((val_yprob_vec, val_yprob))
    val_ypred_vec = np.concatenate((val_ypred_vec, val_ypred))
    val_ytrue_vec = np.concatenate((val_ytrue_vec, y_val))
  # Stack training and validation predictions into two panda df
  train_output = pd.DataFrame({"yprob": train_yprob_vec, "ypred": train_ypred_vec,"ytrue": train_ytrue_vec})
  val_output = pd.DataFrame({"yprob": val_yprob_vec, "ypred": val_ypred_vec,"ytrue": val_ytrue_vec})
  # Evaluate
  train_acc = get_accuracy(train_output["ypred"].values, train_output["ytrue"].values)
  val_acc = get_accuracy(val_output["ypred"].values, val_output["ytrue"].values)
  print(f"\n>> {name} final report")
  print(f"Train CV accuracy: {train_acc:.4}")
  print(f"Valid CV accuracy: {val_acc:.4}")
  # Report time 
  print(f"Duration: {datetime.now() - start_time}")
  return train_output, val_output

svm_train_output, svm_val_output = svm_cv(X_TRAIN, y_TRAIN, model=svm_rbf)

## Performance evaluation

In [ ]:
# Function to quickly add plot details
def plot_details(title = "", xlabel = "X-axis", ylabel = "Y-axis",
                 ax_equal = False, legend = True, leg_loc = None,
                 grid = True, save = False, filename = False):
    # Title and axis
    plt.title(title, fontsize=14)
    plt.xlabel(xlabel, fontsize=13)
    plt.ylabel(ylabel, fontsize=13)
    if ax_equal == True:
        plt.axis('equal') 
    # Legend and grid
    if legend == True:
        legend = plt.legend(frameon = 1, loc = leg_loc, shadow = True)
        frame = legend.get_frame()
        frame.set_facecolor('white')
        frame.set_edgecolor('black')
    if grid == True:
        plt.grid(zorder=0, color="lightgray")
    # Output
    if save == True:
        plt.savefig(filename + ".png", dpi = 300)
    plt.show()

#### Classification report

In [ ]:
print("Random forest:\n", classification_report(rf_val_output["ytrue"], rf_val_output["ypred"]))     
print("LightGBM:\n",classification_report(lgb_val_output["ytrue"], lgb_val_output["ypred"]))
print("SVM RBF:\n",classification_report(svm_val_output["ytrue"], svm_val_output["ypred"]))

#### ROC

In [ ]:
# Compute FPR and TPR
rf_val_auc = roc_auc_score(rf_val_output["ytrue"], rf_val_output["ypred"])
fpr_rf, tpr_rf, _ = roc_curve(rf_val_output["ytrue"], rf_val_output["yprob"])
lgb_val_auc = roc_auc_score(lgb_val_output["ytrue"], lgb_val_output["ypred"])
fpr_lgb, tpr_lgb, _ = roc_curve(lgb_val_output["ytrue"], lgb_val_output["yprob"])
svm_val_auc = roc_auc_score(svm_val_output["ytrue"], svm_val_output["ypred"])
fpr_svm, tpr_svm, _ = roc_curve(svm_val_output["ytrue"], svm_val_output["yprob"])

# Plot ROC
fig, ax = plt.subplots(1, 1, figsize = (6, 4))
ax.plot(fpr_rf, tpr_rf, label=f"RF (AUC = {rf_val_auc:.3})")
ax.plot(fpr_lgb, tpr_lgb, label=f"LGBM (AUC = {lgb_val_auc:.3})")
ax.plot(fpr_svm, tpr_svm, label=f"SVM RBF (AUC = {svm_val_auc:.3})")
ax.plot([0, 1], [0, 1],'r--')
plot_details(title = "ROC on validation data (CV)",
             xlabel = '1 - Specificity (FPR)', 
             ylabel = 'Sensitivity (TPR)',
             leg_loc = "lower right",
             save = True,
             filename = "Plots/roc_rf_lgbm_all_replicate_negFiltered")

#### Plot some metrics

In [ ]:
# RF
rf_train_auc = roc_auc_score(rf_train_output["ytrue"], rf_train_output["ypred"])
rf_train_acc = get_accuracy(rf_train_output["ypred"], rf_train_output["ytrue"])
rf_train_rmse = rmse(rf_train_output["yprob"], rf_train_output["ytrue"])
rf_val_acc = get_accuracy(rf_val_output["ypred"], rf_val_output["ytrue"])
rf_val_rmse = rmse(rf_val_output["yprob"], rf_val_output["ytrue"])
# LGBM
lgb_train_auc = roc_auc_score(lgb_train_output["ytrue"], lgb_train_output["ypred"])
lgb_train_acc = get_accuracy(lgb_train_output["ypred"], lgb_train_output["ytrue"])
lgb_train_rmse = rmse(lgb_train_output["yprob"], lgb_train_output["ytrue"])
lgb_val_acc = get_accuracy(lgb_val_output["ypred"], lgb_val_output["ytrue"])
lgb_val_rmse = rmse(lgb_val_output["yprob"], lgb_val_output["ytrue"])
# SVM
svm_train_auc = roc_auc_score(svm_train_output["ytrue"], svm_train_output["ypred"])
svm_train_acc = get_accuracy(svm_train_output["ypred"], svm_train_output["ytrue"])
svm_train_rmse = rmse(svm_train_output["yprob"], svm_train_output["ytrue"])
svm_val_acc = get_accuracy(svm_val_output["ypred"], svm_val_output["ytrue"])
svm_val_rmse = rmse(svm_val_output["yprob"], svm_val_output["ytrue"])

In [ ]:
train_acc = rf_train_acc, lgb_train_acc, svm_train_acc
train_rmse = rf_train_rmse, lgb_train_rmse, svm_train_rmse
train_auc = rf_train_auc, lgb_train_auc, svm_train_auc

val_acc = rf_val_acc, lgb_val_acc, svm_val_acc
val_rmse = rf_val_rmse, lgb_val_rmse, svm_val_rmse
val_auc = rf_val_auc, lgb_val_auc, svm_val_auc

# Plot
pd_df2 = pd.DataFrame({"Train RMSE": train_rmse, "Train Acc": train_acc, "Train AUC": train_auc,
                       "Val RMSE": val_rmse, "Val Acc": val_acc, "Val AUC": val_auc}, 
                      index = ["RF", "LGBM", "SVM"])

ax = pd_df2.plot(y=["Train RMSE", "Train Acc", "Train AUC", "Val RMSE", "Val Acc", "Val AUC"], 
                 ylim=(0,1.1), figsize=(12,5), fontsize=13,
                 kind="bar", zorder=3, ec ="black",
                 rot=1, width=0.8, color = ["gold", "tab:orange", "tab:red",
                                                       "tab:cyan", "cornflowerblue", "tab:blue"])
# Add details
plt.title("Model performance evaluation", fontsize = 15)
plt.ylabel("Score", fontsize = 13)
plt.grid(axis="y", zorder=0, color="lightgray")  
legend = plt.legend(frameon = 1, shadow = True, bbox_to_anchor=(1.03, 0.6), fontsize=13)
frame = legend.get_frame()
frame.set_facecolor('white')
frame.set_edgecolor('black')
# Annotate scores on top of the bars
for p in ax.patches:
  height = p.get_height()
  ha = {'center': 'center', 'right': 'left', 'left': 'right'}
  xpos='center'
  offset = {'center': 0, 'right': 1, 'left': -1}
  ax.annotate(f"{height:.2}",
              xy=(p.get_x() + p.get_width() / 2, height),
              xytext=(offset[xpos]*3, 3),  
              textcoords="offset points",  
              ha=ha[xpos], va='bottom')
plt.savefig("Plots/models_performance_all_replicate_negFiltered_validation2.png", 
            dpi = 300, bbox_extra_artists=(legend,), bbox_inches='tight')
plt.show()